In [1]:
!cat /proc/cpuinfo | grep flags

flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibr

In [1]:
!pip install apache-tvm

In [2]:
!pip install psutil cloudpickle

In [1]:
!pip install xgboost==1.5.0

In [1]:
import tvm
from tvm import te
from tvm import autotvm
import numpy as np
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner

In [2]:
import logging
import sys
# logging config (for printing tuning log to the screen)
logging.getLogger("autotvm").setLevel(logging.DEBUG)
logging.getLogger("autotvm").addHandler(logging.StreamHandler(sys.stdout))

In [3]:
def run_tuning(tasks, measure_option, tuner="gridsearch", early_stopping=None,
               log_filename="tuning.log", n_trial=None):
    for i, task in enumerate(tasks):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(task, loss_type="rank")
        elif tuner == "ga":
            tuner_obj = GATuner(task, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(task)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(task)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        # do tuning
        n_trial = min(n_trial or len(task.config_space), len(task.config_space))
        tuner_obj.tune(
            n_trial=n_trial or len(task.config_space),
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(n_trial, prefix=prefix),
                autotvm.callback.log_to_file(log_filename),
            ],
        )
def evaluate_best_from_history(logfile, N, L, M, target):
    # apply history best from log file
    with autotvm.apply_history_best(logfile):
        with tvm.target.Target(target):
            s, arg_bufs = matmul_w_filters(N, L, M, "float32")
            func = tvm.build(s, arg_bufs)
    dev = tvm.cpu(0)

    # check correctness
    a_np = np.random.uniform(size=(N, L)).astype(np.float32)
    b_np = np.random.uniform(size=(L, M)).astype(np.float32)
    c_np = np.matmul(a_np, b_np).astype(np.float32)

    c_tvm = tvm.nd.array(np.zeros(c_np.shape, dtype=np.float32), dev)
    func(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm)

    np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-4)


    time_f = func.time_evaluator(func.entry_name, dev, number=10)
    cost = time_f(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm).mean
    print("func", cost*1000)

In [4]:
N,L,M = 512, 512, 512

In [5]:
@autotvm.template("matmul_w_filters")
def matmul_w_filters(N, L, M, candidates=None):
    A = te.placeholder((N, L), name="A", dtype="float32")
    B = te.placeholder((L, M), name="B", dtype="float32")

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    ##### define space begin #####
    cfg = autotvm.get_config()
    filter = lambda v: v.size[0] != 2 and v.size[1] != 2
    multi_filter = lambda e: 32 <= (e["tile_x"].size[1] + e["tile_y"].size[1]) < 128
    cfg.multi_filter(multi_filter)
    candidates = [[4, 128], [8, 64], [16, 32], [32, 16], [64, 8], [128, 4], [2, 256], [256, 2]]

    cfg.define_split("tile_y", y, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    cfg.define_split("tile_x", x, num_outputs=2, policy="candidate", candidate=candidates, filter=filter)
    ##### define space end #####

    # schedule according to config
    yo, yi = cfg["tile_y"].apply(s, C, y)
    xo, xi = cfg["tile_x"].apply(s, C, x)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

In [6]:
t3 = autotvm.task.create("matmul_w_filters", args=(N, L, M), target="llvm")
for idx in range(t3.config_space.range_length):
    if t3.config_space.is_index_valid(idx):
        print(t3.config_space.get(idx))
to3 = {
    "log_filename": "matmul_w_filters.log",
    "n_trial": None,
    "tuner": "xgb",
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.LocalRunner(
            number=1, repeat=10, min_repeat_ms=0, enable_cpu_cache_flush=True
        ),
    ),
}

[('tile_y', [16, 32]), ('tile_x', [8, 64])],None,8
[('tile_y', [32, 16]), ('tile_x', [8, 64])],None,9
[('tile_y', [64, 8]), ('tile_x', [8, 64])],None,10
[('tile_y', [128, 4]), ('tile_x', [8, 64])],None,11
[('tile_y', [8, 64]), ('tile_x', [16, 32])],None,13
[('tile_y', [16, 32]), ('tile_x', [16, 32])],None,14
[('tile_y', [32, 16]), ('tile_x', [16, 32])],None,15
[('tile_y', [64, 8]), ('tile_x', [16, 32])],None,16
[('tile_y', [128, 4]), ('tile_x', [16, 32])],None,17
[('tile_y', [8, 64]), ('tile_x', [32, 16])],None,19
[('tile_y', [16, 32]), ('tile_x', [32, 16])],None,20
[('tile_y', [32, 16]), ('tile_x', [32, 16])],None,21
[('tile_y', [8, 64]), ('tile_x', [64, 8])],None,25
[('tile_y', [16, 32]), ('tile_x', [64, 8])],None,26
[('tile_y', [8, 64]), ('tile_x', [128, 4])],None,31
[('tile_y', [16, 32]), ('tile_x', [128, 4])],None,32


In [7]:
print(t3.config_space)
run_tuning([t3], **to3)
evaluate_best_from_history("matmul_w_filters.log", N, L, M, "llvm")

ConfigSpace (len=16, range_length=36, space_map=
   0 tile_y: Split(policy=candidate, product=512, num_outputs=2) len=6
   1 tile_x: Split(policy=candidate, product=512, num_outputs=2) len=6
)
waiting for device...


DEBUG:autotvm:waiting for device...


device available


DEBUG:autotvm:device available


Get devices for measurement successfully!


...

DEBUG:autotvm:No: 16	GFLOPS: 17.99/21.65	result: MeasureResult(costs=(0.014753978999999999, 0.014759719000000001, 0.01476933, 0.014806109, 0.014874829999999999, 0.015052139, 0.015163929, 0.015223879000000001), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.48999667167663574, timestamp=1713769091.668098)	[('tile_y', [8, 64]), ('tile_x', [16, 32])],None,13


Finish loading 40 records


DEBUG:autotvm:Finish loading 40 records


Cannot find config for target=llvm -keys=cpu , workload=('matmul_w_filters', 512, 512, 512, 'float32'). A fallback configuration is used, which may bring great performance regression.


func 13.444104500000002


In [8]:
t4 = autotvm.task.create("matmul_w_filters", args=(N, L, M), target="llvm -mcpu=core-avx2")
for idx in range(t4.config_space.range_length):
    if t4.config_space.is_index_valid(idx):
        print(t4.config_space.get(idx))
to4 = {
    "log_filename": "matmul_avx2_w_filters.log",
    "n_trial": None,
    "tuner": "xgb",
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.LocalRunner(
            number=1, repeat=10, min_repeat_ms=0, enable_cpu_cache_flush=True
        ),
    ),
}
print(t4.config_space)


[('tile_y', [16, 32]), ('tile_x', [8, 64])],None,8
[('tile_y', [32, 16]), ('tile_x', [8, 64])],None,9
[('tile_y', [64, 8]), ('tile_x', [8, 64])],None,10
[('tile_y', [128, 4]), ('tile_x', [8, 64])],None,11
[('tile_y', [8, 64]), ('tile_x', [16, 32])],None,13
[('tile_y', [16, 32]), ('tile_x', [16, 32])],None,14
[('tile_y', [32, 16]), ('tile_x', [16, 32])],None,15
[('tile_y', [64, 8]), ('tile_x', [16, 32])],None,16
[('tile_y', [128, 4]), ('tile_x', [16, 32])],None,17
[('tile_y', [8, 64]), ('tile_x', [32, 16])],None,19
[('tile_y', [16, 32]), ('tile_x', [32, 16])],None,20
[('tile_y', [32, 16]), ('tile_x', [32, 16])],None,21
[('tile_y', [8, 64]), ('tile_x', [64, 8])],None,25
[('tile_y', [16, 32]), ('tile_x', [64, 8])],None,26
[('tile_y', [8, 64]), ('tile_x', [128, 4])],None,31
[('tile_y', [16, 32]), ('tile_x', [128, 4])],None,32
ConfigSpace (len=16, range_length=36, space_map=
   0 tile_y: Split(policy=candidate, product=512, num_outputs=2) len=6
   1 tile_x: Split(policy=candidate, product=5

In [9]:
run_tuning([t4], **to4)
evaluate_best_from_history("matmul_avx2_w_filters.log", N, L, M, "llvm -mcpu=core-avx2")

waiting for device...


DEBUG:autotvm:waiting for device...


device available


DEBUG:autotvm:device available


Get devices for measurement successfully!


...

DEBUG:autotvm:No: 16	GFLOPS: 23.60/26.76	result: MeasureResult(costs=(0.011320449, 0.011346049, 0.011365079, 0.011367589, 0.011386599, 0.011386990000000001, 0.01140592, 0.011407419), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.38237714767456055, timestamp=1713769142.7419481)	[('tile_y', [16, 32]), ('tile_x', [8, 64])],None,8


Finish loading 21 records


DEBUG:autotvm:Finish loading 21 records


Cannot find config for target=llvm -keys=cpu -mcpu=core-avx2, workload=('matmul_w_filters', 512, 512, 512, 'float32'). A fallback configuration is used, which may bring great performance regression.


func 9.6948807


In [19]:
@autotvm.template("matmul_brute_force_reduced_")
def matmul_brute_force(N, L, M, candidates=None):
    A = te.placeholder((N, L), name="A", dtype="float32")
    B = te.placeholder((L, M), name="B", dtype="float32")

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    ##### define space begin #####
    cfg = autotvm.get_config()
    filter = lambda v: v.size[0] != 1 and v.size[1] != 1
    multi_filter = lambda e: 16 <= (e["tile_x"].size[1] + e["tile_y"].size[1]) < 128
    cfg.multi_filter(multi_filter)

    cfg.define_split("tile_y", y, num_outputs=2, filter=filter)
    cfg.define_split("tile_x", x, num_outputs=2, filter=filter)
    ##### define space end #####

    # schedule according to config
    yo, yi = cfg["tile_y"].apply(s, C, y)
    xo, xi = cfg["tile_x"].apply(s, C, x)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

In [20]:
t2 = autotvm.task.create("matmul_brute_force_reduced_", args=(N, L, M), target="llvm")
for idx in range(t2.config_space.range_length):
    if t2.config_space.is_index_valid(idx):
        print(t2.config_space.get(idx))
to2 = {
    "log_filename": "matmul_brute_force_reduced_.log",
    "n_trial": None,
    "tuner": "xgb",
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.LocalRunner(
            number=1, repeat=10, min_repeat_ms=0, enable_cpu_cache_flush=True
        ),
    ),
}

[('tile_y', [-1, 16]), ('tile_x', [-1, 2])],None,3
[('tile_y', [-1, 32]), ('tile_x', [-1, 2])],None,4
[('tile_y', [-1, 64]), ('tile_x', [-1, 2])],None,5
[('tile_y', [-1, 16]), ('tile_x', [-1, 4])],None,12
[('tile_y', [-1, 32]), ('tile_x', [-1, 4])],None,13
[('tile_y', [-1, 64]), ('tile_x', [-1, 4])],None,14
[('tile_y', [-1, 8]), ('tile_x', [-1, 8])],None,20
[('tile_y', [-1, 16]), ('tile_x', [-1, 8])],None,21
[('tile_y', [-1, 32]), ('tile_x', [-1, 8])],None,22
[('tile_y', [-1, 64]), ('tile_x', [-1, 8])],None,23
[('tile_y', [-1, 2]), ('tile_x', [-1, 16])],None,27
[('tile_y', [-1, 4]), ('tile_x', [-1, 16])],None,28
[('tile_y', [-1, 8]), ('tile_x', [-1, 16])],None,29
[('tile_y', [-1, 16]), ('tile_x', [-1, 16])],None,30
[('tile_y', [-1, 32]), ('tile_x', [-1, 16])],None,31
[('tile_y', [-1, 64]), ('tile_x', [-1, 16])],None,32
[('tile_y', [-1, 2]), ('tile_x', [-1, 32])],None,36
[('tile_y', [-1, 4]), ('tile_x', [-1, 32])],None,37
[('tile_y', [-1, 8]), ('tile_x', [-1, 32])],None,38
[('tile_y', [

In [17]:
print(t2.config_space)

ConfigSpace (len=33, range_length=100, space_map=
   0 tile_y: Split(policy=factors, product=512, num_outputs=2) len=10
   1 tile_x: Split(policy=factors, product=512, num_outputs=2) len=10
)


In [21]:

run_tuning([t2], **to2)

waiting for device...


DEBUG:autotvm:waiting for device...


device available


DEBUG:autotvm:device available


Get devices for measurement successfully!


...

DEBUG:autotvm:No: 25	GFLOPS: 14.81/22.01	result: MeasureResult(costs=(0.017960889, 0.01798026, 0.017995629000000003, 0.018074989, 0.018155018999999998, 0.018163669, 0.01819938, 0.018480989), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.4337656497955322, timestamp=1713769605.0771787)	[('tile_y', [-1, 4]), ('tile_x', [-1, 32])],None,37


No: 26	GFLOPS: 6.11/22.01	result: MeasureResult(costs=(0.043686748000000004, 0.043818538, 0.043882579, 0.043890509, 0.044000238000000004, 0.044026569, 0.044054948000000003, 0.044083849), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.7779927253723145, timestamp=1713769605.8342779)	[('tile_y', [-1, 8]), ('tile_x', [-1, 16])],None,29


DEBUG:autotvm:No: 26	GFLOPS: 6.11/22.01	result: MeasureResult(costs=(0.043686748000000004, 0.043818538, 0.043882579, 0.043890509, 0.044000238000000004, 0.044026569, 0.044054948000000003, 0.044083849), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.7779927253723145, timestamp=1713769605.8342779)	[('tile_y', [-1, 8]), ('tile_x', [-1, 16])],None,29


No: 27	GFLOPS: 0.63/22.01	result: MeasureResult(costs=(0.349120528, 0.356044087, 0.369796607, 0.43288926400000005, 0.458574844, 0.471594323, 0.47427137399999997, 0.475638714), error_no=MeasureErrorNo.NO_ERROR, all_cost=4.8864734172821045, timestamp=1713769610.8383458)	[('tile_y', [-1, 64]), ('tile_x', [-1, 2])],None,5


DEBUG:autotvm:No: 27	GFLOPS: 0.63/22.01	result: MeasureResult(costs=(0.349120528, 0.356044087, 0.369796607, 0.43288926400000005, 0.458574844, 0.471594323, 0.47427137399999997, 0.475638714), error_no=MeasureErrorNo.NO_ERROR, all_cost=4.8864734172821045, timestamp=1713769610.8383458)	[('tile_y', [-1, 64]), ('tile_x', [-1, 2])],None,5


In [22]:
# apply history best from log file
with autotvm.apply_history_best("matmul_brute_force_reduced_.log"):
    with tvm.target.Target("llvm"):
        s, arg_bufs = matmul_brute_force(N, L, M, "float32")
        func = tvm.build(s, arg_bufs)
dev = tvm.cpu(0)

# check correctness
a_np = np.random.uniform(size=(N, L)).astype(np.float32)
b_np = np.random.uniform(size=(L, M)).astype(np.float32)
c_np = np.matmul(a_np, b_np).astype(np.float32)

c_tvm = tvm.nd.array(np.zeros(c_np.shape, dtype=np.float32), dev)
func(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm)

np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-4)


time_f = func.time_evaluator(func.entry_name, dev, number=10)
cost = time_f(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm).mean
print("func", cost*1000)

Finish loading 27 records


DEBUG:autotvm:Finish loading 27 records


Cannot find config for target=llvm -keys=cpu , workload=('matmul_brute_force_reduced_', 512, 512, 512, 'float32'). A fallback configuration is used, which may bring great performance regression.


func 159.2494124
